In [1]:
%pip install pandas

In [2]:
import pandas as pd

In [3]:
# Replace the URL below with your GitHub raw file URL
url = "https://raw.githubusercontent.com/IKRAMJAAFAR/DeeL/refs/heads/main/dataset.csv?token=GHSAT0AAAAAACZZJM4A75DJZ7QS6RRPDJOOZZDRB5A"

# Load the dataset directly into a DataFrame
dataframe = pd.read_csv(url)
# Display the first few rows of the dataframe
dataframe

,Unnamed: 0,timestamp,value
0,0,2014-07-01 00:00:00,10844
1,1,2014-07-01 00:30:00,8127
2,2,2014-07-01 01:00:00,6210
3,3,2014-07-01 01:30:00,4656
4,4,2014-07-01 02:00:00,3820
...,...,...,...
10315,10315,2015-01-31 21:30:00,24670
10316,10316,2015-01-31 22:00:00,25721
10317,10317,2015-01-31 22:30:00,27309
10318,10318,2015-01-31 23:00:00,26591
